Restrict to US 

In [1]:
%%configure -f
{
    "driverMemory": "20g",
    "executorMemory": "8g",
    "numExecutors": 5,
    "executorCores": 8,
    "conf": 
        {"spark.yarn.executor.memoryOverhead":"2g"}
    }

In [2]:
import datetime

import pyspark.sql.functions as F
import pyspark.sql.types as T 
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1667605665011_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# bootstrapped libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# the ask

I think this is what is being asked:
- coverage if we just look at Priceline
    - of the markets Priceline serves, what is the num request, num solutoins
- also look at:
    - frequency of advanced purchases "across the board" (I take this to mean not just Priceline?)
    - frequency of stay durations
- OK to limit to US for now

In [4]:
AIRPORT_LOOKUP_PATH = "s3://tvlp-ds-users/kendra-frederick/reference-data/AIRPORT.CSV"
airport_df = spark.read.option("header", "true").csv(AIRPORT_LOOKUP_PATH)

pl_pccs_df = spark.read.csv("s3://tvlp-ds-users/kendra-frederick/reference-data/priceline_PCCs_for_kendra.csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# pl_pccs_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---+-------+----+-------+----------------------+--------------------+-----------------------+------------------------+----------------+--------------------------+----------------+
|  Obs|CRS|CRS_PCC| PCC|   IATA|Branch_Customer_Number|       Agency_Branch|Agency_National_Account|Raw_Master_Customer_Name|Raw_Country_Code|Raw_Master_Customer_Number|Raw_Country_Name|
+-----+---+-------+----+-------+----------------------+--------------------+-----------------------+------------------------+----------------+--------------------------+----------------+
|11679| 1G|1G_5RD7|5RD7| 236466|                625917|Cvfr Consolidatio...|          Priceline.Com|           Priceline.Com|              US|                    624055|   United States|
|29855| 1G| 1G_4D6| 4D6|3430106|                622919|Richmond Intl Trv...|          Priceline.Com|           Priceline.Com|              US|                    624055|   United States|
|30856| 1G|1G_7E8M|7E8M|1633544|                625030|     Price

# load data

In [5]:
BASE_DATA_DIR = "s3://tvlp-ds-air-shopping-pn/v1_5"

# take just an hour's worth of data for now
date = datetime.date(2022,11,1)
hour = 11
date_str = date.strftime("%Y%m%d")

input_path = "{}/year={}/month={}/day={}/hour={:02d}/".format(BASE_DATA_DIR, date.year, date.month, date_str, hour)

df = spark.read.parquet(input_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# # restrict to round-trips and/or one-ways
# cond_rt = (((F.col("out_origin_airport") == F.col("in_destination_airport")) &
#         (F.col("out_destination_airport") == F.col("in_origin_airport"))))
# cond_ow = ((F.col("in_origin_airport").isNull()) & (F.col("in_destination_airport").isNull()))

# df_filt = df_raw.filter(cond_rt | cond_ow)
# df_filt = df_filt.withColumn("round_trip", 
#     F.when(cond_rt, 1).otherwise(0))

In [6]:
c_all = df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# restrict to US for now
pos = 'US'
currency = 'USD'
df = df.filter(
        (F.col("point_of_sale") == pos) & (F.col("currency") == currency)
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# poorly named variable
# c0 = df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# c0, c0/c_all

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(282163174, 0.10063777940258582)

In [8]:
df = (df
                .join(
                    airport_df.select('airport_code', 'country_code'),
                    on=[df['out_origin_city'] == airport_df['airport_code']])
                .withColumnRenamed("country_code", "origin_country")
                .drop("airport_code")
            )
df = (df
            .join(
                airport_df.select('airport_code', 'country_code'),
                on=[df['out_destination_city'] == airport_df['airport_code']])
            .withColumnRenamed("country_code", "destination_country")
            .drop("airport_code")
        )

# filter on O or D in POS
df = df.filter(
        (F.col('origin_country') == pos) | (F.col("destination_country") == pos)
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
c0 = df.count()
c0, c0/c_all

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(181965041, 0.06490059420419081)

In [10]:
c_all_us = c0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# add columns to assist with joining and grouping
df = (df
      .withColumn("market", F.concat_ws("-", F.col("out_origin_airport"), F.col("out_destination_airport")))
      .withColumn("gds_pcc", F.concat_ws("_", F.col("gds"), F.col("pcc")))
     )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# filter on Priceline PCCs
df_pl = df.join(pl_pccs_df.select('CRS_PCC'), on=[df['gds_pcc'] == pl_pccs_df['CRS_PCC']])

# num solutions from Priceline PCCs
c_pl = df_pl.count()
c_pl

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

41091462

In [13]:
# c_pl / c0
c_pl / 181965041

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.2258206399107178

In [84]:
c0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

181965041

In [12]:
pl_markets = df_pl.select("market").distinct()
df_pl_markets = df.join(pl_markets, on="market")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
m0 = df.select("market").distinct().count()
m2 = pl_markets.count()
c2 = df_pl_markets.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
m2, m0, c2, c2/c_all_us

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(21325, 39283, 168629060, 0.9267113016505187)

In [15]:
m2/ m0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5428556882111855

Numbers-wise, we appear to have good coverage

This also answers: overall market size

# "coverage"
For markets served by Priceline:
- num requests, num solutions

In [14]:
uniq_key_cols = ["id", "originalRequest", "shop_req_timeStamp", "gds", "pcc"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
srch_0 = df.select(uniq_key_cols).distinct().count()
srch_pl_m = df_pl_markets.select(uniq_key_cols).distinct().count()
srch_0, srch_pl_m

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(775933, 702292)

In [18]:
srch_pl_m/srch_0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9050936098864206

In [19]:
# avg num solutions per search
c2 / srch_pl

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

240.11246034413037

In [15]:
# num searches from Priceline PCCs
srch_pl = df_pl.select(uniq_key_cols).distinct().count()
srch_pl

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

169582

In [16]:
srch_pl / 775933

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.21855237501176003

## Distribution - volume by market

In [27]:
pl_market_stats = (df_pl_markets
                   .groupBy("market")
                   .agg(
                       # note id alone does not identify a unique search
                       # F.countDistinct("id").alias("unique_ids"),
                       F.countDistinct(*uniq_key_cols).alias("num_searches"),
                       F.count("id").alias("num_solutions"), # could count any column, really
                   )
                  )

# pl_market_stats = pl_market_stats.cache()

pl_market_stats = (pl_market_stats
                   .withColumn("search_rank", F.row_number().over(Window.partitionBy().orderBy(F.desc("num_searches"))))
                   # .withColumn("search_solutions", F.row_number().over(Window.partitionBy().orderBy(F.desc("num_solutions"))))
                   # next time correct name of column:
                   .withColumn("solution_rank", F.row_number().over(Window.partitionBy().orderBy(F.desc("num_solutions"))))
                  )

w = Window.partitionBy()
pl_market_stats = (pl_market_stats
                   .withColumn("pct_searches", F.col("num_searches") / F.sum("num_searches").over(w))
                   .withColumn("pct_solutions", F.col("num_solutions") / F.sum("num_solutions").over(w))                  
                  )

w2 = Window.partitionBy().orderBy("search_rank").rowsBetween(Window.unboundedPreceding, Window.currentRow)
w3 = Window.partitionBy().orderBy("solution_rank").rowsBetween(Window.unboundedPreceding, Window.currentRow)

pl_market_stats = (pl_market_stats
                   .withColumn("cum_pct_searches", F.sum("pct_searches").over(w2))
                   .withColumn("cum_pct_solutions", F.sum("pct_solutions").over(w3))
                  )

In [29]:
pl_market_stats.orderBy("search_rank").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+-------------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+
| market|num_searches|num_solutions|search_rank|solution_rank|        pct_searches|       pct_solutions|    cum_pct_searches|   cum_pct_solutions|
+-------+------------+-------------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+
|SFO-LAX|       10070|      2032882|          1|            1|0.008629111038749592|0.012055347992807408|0.008629111038749592|0.012055347992807408|
|LAX-MNL|        5271|      1286449|          2|            3| 0.00451678692008432|0.007628868950583014|0.013145897958833912|0.027490943731762488|
|SFO-SNA|        4708|       862844|          3|            9|0.004034345061612...|0.005116816757443...|0.017180243020445934| 0.06147179495633791|
|SJC-LAX|        4632|       870513|          4|            8|0.003969219695281839|0.005162295276982508|0.021149462715

In [ ]:
ms_pdf = pl_market_stats.toPandas()

In [ ]:
pd.set_option("display.max_columns", 100)

In [51]:
len(ms_pdf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

21325

In [79]:
top_n = 100
data_col = 'unique_key'
data = ms_pdf[:top_n][data_col]
xs = range(top_n)
plt.bar(xs, data)
plt.ylabel("num searches")
plt.xlabel("market");
plt.show()
# plt.savefig("s3://tvlp-ds-users/kendra-frederick/tmp/test-fig.png")
plt.savefig("test-fig.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
top_n = 100
data_col = 'num_solutions'
data = ms_pdf.sort_values(by=data_col, ascending=False)[:top_n][data_col]
xs = range(top_n)
plt.figure(figsize=(8,5))
plt.bar(xs, data)
plt.ylabel("num_solutions")
plt.xlabel("market");
plt.savefig("/tmp/num_solutions.png")
# plt.savefig("test-fig.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
top_n = 1000

bar_data_col = 'unique_key'
line_data_col = 'cum_pct_searches'
order_col = 'search_rank'

data_df = ms_pdf.sort_values(by=order_col)[:top_n]
data_bar = data_df[bar_data_col]
data_line = data_df[line_data_col]
xs = np.arange(len(data_bar))

fig, ax = plt.subplots(figsize=(10,5))
ax.bar(xs, data_bar);
ax.set_ylabel("num searches")

ax2 = ax.twinx()
ax2.plot(xs, data_line, marker=".", color="red")
ax2.set_ylim(0, 1)
ax2.set_ylabel("Cumulative % Total Searches")

plt.savefig("/tmp/search-overlay.png")
# plt.savefig("test-fig.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
m2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

21325

In [100]:
num_pct = 0.01
abs_num = int(m2 * num_pct)
print(f"For the top {num_pct*100}% of markets:")
print(f"Number of markets: {abs_num:,}")
x = ms_pdf[ms_pdf['search_rank'] == abs_num]
cum_pct = x['cum_pct_searches'].values[0]
print(f"Cumulative % search volume: {cum_pct*100:.0f}%")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For the top 1.0% of markets:
Number of markets: 213
Cumulative % search volume: 31%

In [102]:
num_pct = 0.05
abs_num = int(m2 * num_pct)
print(f"For the top {num_pct*100}% of markets:")
print(f"Number of markets: {abs_num:,}")
x = ms_pdf[ms_pdf['search_rank'] == abs_num]
cum_pct = x['cum_pct_searches'].values[0]
print(f"Cumulative % search volume: {cum_pct*100:.0f}%")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For the top 5.0% of markets:
Number of markets: 1,066
Cumulative % search volume: 63%

In [101]:
num_pct = 0.1
abs_num = int(m2 * num_pct)
print(f"For the top {num_pct*100}% of markets:")
print(f"Number of markets: {abs_num:,}")
x = ms_pdf[ms_pdf['search_rank'] == abs_num]
cum_pct = x['cum_pct_searches'].values[0]
print(f"Cumulative % search volume: {cum_pct*100:.0f}%")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For the top 10.0% of markets:
Number of markets: 2,132
Cumulative % search volume: 77%

In [103]:
top_n = 1000

bar_data_col = 'num_solutions'
line_data_col = 'cum_pct_solutions'
order_col = 'search_solutions'

data_df = ms_pdf.sort_values(by=order_col)[:top_n]
data_bar = data_df[bar_data_col]
data_line = data_df[line_data_col]
xs = np.arange(len(data_bar))

fig, ax = plt.subplots(figsize=(10,5))
ax.bar(xs, data_bar);
ax.set_ylabel("num solutions")

ax2 = ax.twinx()
ax2.plot(xs, data_line, marker=".", color="red")
ax2.set_ylim(0, 1)
ax2.set_ylabel("Cumulative % Total Solutions")

plt.savefig("/tmp/solutions-overlay.png")
# plt.savefig("test-fig.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
num_pct = 0.01
abs_num = int(m2 * num_pct)
print(f"For the top {num_pct*100}% of markets:")
print(f"Number of markets: {abs_num:,}")
x = ms_pdf[ms_pdf[order_col] == abs_num]
cum_pct = x[line_data_col].values[0]
print(f"Cumulative % search volume: {cum_pct*100:.0f}%")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For the top 1.0% of markets:
Number of markets: 213
Cumulative % search volume: 35%

In [105]:
num_pct = 0.05
abs_num = int(m2 * num_pct)
print(f"For the top {num_pct*100}% of markets:")
print(f"Number of markets: {abs_num:,}")
x = ms_pdf[ms_pdf[order_col] == abs_num]
cum_pct = x[line_data_col].values[0]
print(f"Cumulative % search volume: {cum_pct*100:.0f}%")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For the top 5.0% of markets:
Number of markets: 1,066
Cumulative % search volume: 66%

In [106]:
num_pct = 0.1
abs_num = int(m2 * num_pct)
print(f"For the top {num_pct*100}% of markets:")
print(f"Number of markets: {abs_num:,}")
x = ms_pdf[ms_pdf[order_col] == abs_num]
cum_pct = x[line_data_col].values[0]
print(f"Cumulative % search volume: {cum_pct*100:.0f}%")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For the top 10.0% of markets:
Number of markets: 2,132
Cumulative % search volume: 80%

# Distribution

In [33]:
# convert dates (which are ints) to datetime
df_pl_markets.registerTempTable("data")
df_pl_markets = spark.sql("""
    SELECT *,
        TO_DATE(CAST(UNIX_TIMESTAMP(CAST(out_departure_date AS string), 'yyyyMMdd') AS TIMESTAMP)) AS outDeptDt_dt,
        TO_DATE(CAST(UNIX_TIMESTAMP(CAST(in_departure_date AS string), 'yyyyMMdd') AS TIMESTAMP)) AS inDeptDt_dt
    FROM data
""")

df_pl_markets = (df_pl_markets
                 .withColumn("searchDt_dt", F.lit(date))
                )

# do date math
df_pl_markets = (df_pl_markets
    .withColumn('days_til_dept', F.datediff(
                    F.col('outDeptDt_dt'), F.col('searchDt_dt')))
    .withColumn('stay_duration', F.datediff(
                    F.col('inDeptDt_dt'), F.col('outDeptDt_dt')))
)

## advanced purchase

In [34]:
dtd_stats = df_pl_markets.groupBy("days_til_dept").agg(
    F.countDistinct("market").alias("num_markets"),
    F.countDistinct(*uniq_key_cols).alias("num_searches"),
    F.count("id").alias("num_solutions"), # could count any column, really
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
dtd_stats = dtd_stats.orderBy("days_til_dept")

In [39]:
w = Window.partitionBy()
dtd_stats = (dtd_stats
             .withColumn("pct_searches", F.col("num_searches") / F.sum("num_searches").over(w))
             .withColumn("pct_solutions", F.col("num_solutions") / F.sum("num_solutions").over(w))                  
            )

w2 = Window.partitionBy().orderBy("days_til_dept").rowsBetween(Window.unboundedPreceding, Window.currentRow)
# w3 = Window.partitionBy().orderBy("solution_rank").rowsBetween(Window.unboundedPreceding, Window.currentRow)

dtd_stats = (dtd_stats
                   .withColumn("cum_pct_searches", F.sum("pct_searches").over(w2))
                   .withColumn("cum_pct_solutions", F.sum("pct_solutions").over(w2))
                  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
dtd_stats.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------+------------+-------------+--------------------+--------------------+--------------------+--------------------+
|days_til_dept|num_markets|num_searches|num_solutions|        pct_searches|       pct_solutions|    cum_pct_searches|   cum_pct_solutions|
+-------------+-----------+------------+-------------+--------------------+--------------------+--------------------+--------------------+
|            0|       2062|        6232|       798869|0.008873801780455992|0.004737433749556571|0.008873801780455992|0.004737433749556571|
|            1|       2463|       10555|      2086122|0.015029361006532896| 0.01237107056162206| 0.02390316278698889| 0.01710850431117863|
|            2|       2494|       11115|      2789450|0.015826750126727913|0.016541929368520468|  0.0397299129137168|0.033650433679699096|
|            3|       3130|       13521|      3439064|0.019252675525280084|0.020394254703192914| 0.05898258843899688| 0.05404468838289201|
|            4|       2413|

In [45]:
data_df = dtd_stats.toPandas()

In [47]:
bar_data_col = 'num_searches'
line_data_col = 'cum_pct_searches'
order_col = 'days_til_dept'
x_label = 'advanced purchase (days)'
top_n = 1000
what = 'Searches'
title = 'Distribution of Searches by Advanced Purchase'
file_name = 'dtd-overlay'

data_df = data_df.sort_values(by=order_col)[:top_n]
data_bar = data_df[bar_data_col]
data_line = data_df[line_data_col]
xs = range(len(data_bar))

fig, ax = plt.subplots(figsize=(10,5))
ax.bar(xs, data_bar);
ax.set_ylabel(f"Num {what}")
ax.set_xlabel(x_label)

ax2 = ax.twinx()
ax2.plot(xs, data_line, marker=".", color="red")
ax2.set_ylim(0, 1)
ax2.set_ylabel(f"Cumulative % Total {what}")

fig.suptitle(title)
plt.savefig(f"/tmp/{file_name}.png")
# plt.savefig("test-fig.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
for num in [14, 30, 60, 90, 120]:
    x = data_df[data_df[order_col] == num]
    cum_pct = x[line_data_col].values[0]
    print(f"At {num} {x_label}:")
    print(f"Cumulative % search volume: {cum_pct*100:.0f}%")
    print()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

At 14 advanced purchase (days):
Cumulative % search volume: 22%

At 30 advanced purchase (days):
Cumulative % search volume: 41%

At 60 advanced purchase (days):
Cumulative % search volume: 70%

At 90 advanced purchase (days):
Cumulative % search volume: 83%

At 120 advanced purchase (days):
Cumulative % search volume: 89%

In [50]:
pd.set_option("display.max_columns", 100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
data_df.sort_values(by=bar_data_col, ascending=False).head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    days_til_dept  num_markets  num_searches  num_solutions  pct_searches  \
7               7         3062         18634        5858833      0.026533   
30             30         2706         18222        4249684      0.025946   
3               3         3130         13521        3439064      0.019253   
22             22         2775         11760        3216119      0.016745   
16             16         3193         11695        2752913      0.016653   

    pct_solutions  cum_pct_searches  cum_pct_solutions  
7        0.034744          0.126708           0.122821  
30       0.025201          0.413154           0.392398  
3        0.020394          0.058983           0.054045  
22       0.019072          0.321754           0.312693  
16       0.016325          0.243053           0.236102

## stay duration (LOS)

In [38]:
los_stats = df_pl_markets.groupBy("stay_duration").agg(
    F.countDistinct("market").alias("num_markets"),
    F.countDistinct(*uniq_key_cols).alias("num_searches"),
    F.count("id").alias("num_solutions"), # could count any column, really
).orderBy("stay_duration")
los_stats.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------+------------+-------------+
|stay_duration|num_markets|num_searches|num_solutions|
+-------------+-----------+------------+-------------+
|         null|      20265|      337062|     16354294|
|            0|       1393|        4246|       470257|
|            1|       1529|        6198|      2868796|
|            2|       2954|       16865|      8787847|
|            3|       4429|       36957|     17147188|
|            4|       4619|       32821|     15541942|
|            5|       3646|       25112|     11866367|
|            6|       3216|       17881|      7804802|
|            7|       5044|       47266|     21171908|
|            8|       2751|       16110|      6367593|
|            9|       2331|       14173|      5528270|
|           10|       2095|       12392|      4854014|
|           11|       1643|        8410|      3301193|
|           12|       1351|        6236|      2613759|
|           13|       1311|        6669|      2521226|
|         

Save data to HDFS; download & plot locally

In [52]:
rt_los_stats = los_stats.dropna()

w = Window.partitionBy()
rt_los_stats = (rt_los_stats
             .withColumn("pct_searches", F.col("num_searches") / F.sum("num_searches").over(w))
             .withColumn("pct_solutions", F.col("num_solutions") / F.sum("num_solutions").over(w))                  
            )

w2 = Window.partitionBy().orderBy("stay_duration").rowsBetween(Window.unboundedPreceding, Window.currentRow)
# w3 = Window.partitionBy().orderBy("solution_rank").rowsBetween(Window.unboundedPreceding, Window.currentRow)

rt_los_stats = (rt_los_stats
                   .withColumn("cum_pct_searches", F.sum("pct_searches").over(w2))
                   .withColumn("cum_pct_solutions", F.sum("pct_solutions").over(w2))
                  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
rt_los_stats.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------+------------+-------------+--------------------+--------------------+--------------------+--------------------+
|stay_duration|num_markets|num_searches|num_solutions|        pct_searches|       pct_solutions|    cum_pct_searches|   cum_pct_solutions|
+-------------+-----------+------------+-------------+--------------------+--------------------+--------------------+--------------------+
|            0|       1393|        4246|       470257|0.011575506598802101|0.003088213578341667|0.011575506598802101|0.003088213578341667|
|            1|       1529|        6198|      2868796| 0.01689707722547702|0.018839602091393133| 0.02847258382427912|  0.0219278156697348|
|            2|       2954|       16865|      8787847|0.045977606874422386| 0.05771046136429459| 0.07445019069870151|  0.0796382770340294|
|            3|       4429|       36957|     17147188| 0.10075270781251278| 0.11260689115096062|  0.1752028985112143| 0.19224516818499002|
|            4|       4619|

In [55]:
los_stats_pdf = rt_los_stats.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
data_df = los_stats_pdf.copy()
bar_data_col = 'num_searches'
line_data_col = 'cum_pct_searches'
order_col = 'stay_duration'
x_label = 'stay duration (days)'
top_n = 60
what = 'Searches'
title = 'Distribution of Searches by Stay Duration'
file_name = 'los-overlay'

data_df = data_df.sort_values(by=order_col)[:top_n]
data_bar = data_df[bar_data_col]
data_line = data_df[line_data_col]
xs = range(len(data_bar))

fig, ax = plt.subplots(figsize=(10,5))
ax.bar(xs, data_bar);
ax.set_ylabel(f"Num {what}")
ax.set_xlabel(x_label)

ax2 = ax.twinx()
ax2.plot(xs, data_line, marker=".", color="red")
ax2.set_ylim(0, 1)
ax2.set_ylabel(f"Cumulative % Total {what}")

fig.suptitle(title)
plt.savefig(f"/tmp/{file_name}.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
los_stats_pdf.sort_values(by=bar_data_col, ascending=False).head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   stay_duration  num_markets  num_searches  num_solutions  pct_searches  \
7              7         5044         47266       21171908      0.128857   
3              3         4429         36957       17147188      0.100753   
4              4         4619         32821       15541942      0.089477   
5              5         3646         25112       11866367      0.068461   
6              6         3216         17881        7804802      0.048747   

   pct_solutions  cum_pct_searches  cum_pct_solutions  
7       0.139038          0.510745           0.562530  
3       0.112607          0.175203           0.192245  
4       0.102065          0.264680           0.294310  
5       0.077927          0.333141           0.372238  
6       0.051255          0.381888           0.423492

In [64]:
data_df.tail(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    stay_duration  num_markets  num_searches  num_solutions  pct_searches  \
59             59          107           331         139951      0.000902   

    pct_solutions  cum_pct_searches  cum_pct_solutions  
59       0.000919          0.969488           0.969049

In [65]:
for num in [3, 7, 14, 21, 30]:
    x = data_df[data_df[order_col] == num]
    pct = x['pct_searches'].values[0]
    cum_pct = x[line_data_col].values[0]
    print(f"At {num} {x_label}:")
    print(f"% search volume: {pct*100:.1f}%")
    print(f"Cumulative % search volume: {cum_pct*100:.1f}%")
    print()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

At 3 stay duration (days):
% search volume: 10.1%
Cumulative % search volume: 17.5%

At 7 stay duration (days):
% search volume: 12.9%
Cumulative % search volume: 51.1%

At 14 stay duration (days):
% search volume: 3.7%
Cumulative % search volume: 72.2%

At 21 stay duration (days):
% search volume: 1.9%
Cumulative % search volume: 82.8%

At 30 stay duration (days):
% search volume: 0.9%
Cumulative % search volume: 90.4%

# Heatmap -flying blind

In [29]:
pl_market_stats.orderBy("search_rank").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+-------------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+
| market|num_searches|num_solutions|search_rank|solution_rank|        pct_searches|       pct_solutions|    cum_pct_searches|   cum_pct_solutions|
+-------+------------+-------------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+
|SFO-LAX|       10070|      2032882|          1|            1|0.008629111038749592|0.012055347992807408|0.008629111038749592|0.012055347992807408|
|LAX-MNL|        5271|      1286449|          2|            3| 0.00451678692008432|0.007628868950583014|0.013145897958833912|0.027490943731762488|
|SFO-SNA|        4708|       862844|          3|            9|0.004034345061612...|0.005116816757443...|0.017180243020445934| 0.06147179495633791|
|SJC-LAX|        4632|       870513|          4|            8|0.003969219695281839|0.005162295276982508|0.021149462715

In [68]:
# restrict to round-trips and/or one-ways
cond_rt = (((F.col("out_origin_airport") == F.col("in_destination_airport")) &
        (F.col("out_destination_airport") == F.col("in_origin_airport"))))
cond_ow = ((F.col("in_origin_airport").isNull()) & (F.col("in_destination_airport").isNull()))

# df_pl_markets = df_pl_markets.filter(cond_rt | cond_ow)
df_pl_markets = df_pl_markets.withColumn("trip_type", 
    F.when(cond_rt, "rt").otherwise(F.when(cond_ow, "ow").otherwise("other")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
market = 'SFO-LAX'
market_df = df_pl_markets.filter(F.col("market") == market)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
market_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['market', 'id', 'group_id', 'originalRequest', 'shop_req_timeStamp', 'gds', 'pcc', 'point_of_sale', 'constricted_search', 'split_ticket', 'lowest_fare_not_in_result', 'request_PTC', 'robotic_shop_type', 'confidence_level', 'pos_city', 'private_fare', 'currency', 'fare', 'tax', 'validating_cxr', 'fare_break_down_by_PTC', 'tax_break_down_by_PTC', 'tax_breakdown', 'refund_rule', 'response_PTC', 'channel_id', 'private_fare_account_code', 'fare_construction_text', 'out_durations', 'out_trip_duration_time', 'out_layovers', 'out_origin_airport', 'out_destination_airport', 'out_origin_city', 'out_destination_city', 'out_departure_date', 'out_departure_time', 'out_arrival_date', 'out_arrival_time', 'out_departure_time_zone', 'out_arrival_time_zone', 'out_departure_epoc', 'out_arrival_epoc', 'out_departure_terminal', 'out_arrival_terminal', 'out_airport_search', 'out_marketing_cxr', 'out_operating_cxr', 'out_flight_numbers', 'out_booking_class', 'out_cabin_class', 'out_seats', 'out_avail_source

In [71]:
market_df_rt = market_df.filter(F.col("trip_type") == "rt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
market_grp_df = market_df_rt.groupBy("out_departure_date", "stay_duration").agg(
                        F.countDistinct(*uniq_key_cols).alias("num_searches"),
                       F.count("id").alias("num_solutions"), # could count any column, really
                   )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
market_pdf = market_grp_df.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
market_pdf.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   out_departure_date  stay_duration  num_searches  num_solutions
0            20221212             63             1            628
1            20221128             14             1            318
2            20221119             75             1            424
3            20221211             76             2            704
4            20230118             13             2            882

In [76]:
market_pivot = market_pdf.pivot(index="stay_duration", 
                                columns="out_departure_date", 
                                values="num_searches")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
plt.figure(figsize=(12, 12))
sns.heatmap(market_pivot, cmap='Greens', square=True,
                        vmin=0,
                        cbar_kws={'label': 'num searches',
                                'shrink': 0.5});
plt.title(f"{market} shop volume");
plt.ylabel("stay duration (days)")
plt.xlabel("departure date");
plt.savefig(f"/tmp/{market}-heatmap.png")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Clip data

In [80]:
market_pdf_clip = market_pdf[(market_pdf['stay_duration'] <= 30)]
market_pivot = market_pdf_clip.pivot(index="stay_duration", 
                                columns="out_departure_date", 
                                values="num_searches")
plt.figure(figsize=(15, 6))
sns.heatmap(market_pivot, cmap='Greens', square=True,
                        vmin=0,
                        cbar_kws={'label': 'num searches',
                                'shrink': 0.5});
plt.title(f"{market} shop volume");
plt.ylabel("stay duration (days)")
plt.xlabel("departure date");
plt.savefig(f"/tmp/{market}-heatmap.png")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…